In [3]:
import mlflow
mlflow.set_experiment("Review_sentiment")

2024/03/29 00:45:33 INFO mlflow.tracking.fluent: Experiment with name 'Review_sentiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///e:/Internship_2024/Backend_data/ml_flow/mlruns/695524463574134291', creation_time=1711653333152, experiment_id='695524463574134291', last_update_time=1711653333152, lifecycle_stage='active', name='Review_sentiment', tags={}>

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [18]:
df = pd.read_csv(r"E:\Internship_2024\Backend_data\ml_flow\data.csv")

In [12]:
df.head(5)

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [20]:
new_df=df[['Review text',"Ratings"]]

In [24]:
new_df.dropna(inplace=True)

In [25]:
new_df.isna().sum()

Review text    0
Ratings        0
dtype: int64

In [26]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

In [27]:
import warnings

warnings.filterwarnings('ignore')

In [29]:
X=new_df['Review text']
y=new_df['Ratings']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

print(X_train.shape, X_test.shape)

(6382,) (2128,)


In [31]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Define pipeline steps
pipe_1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', KNeighborsClassifier())
])
# Observe the Key Value Pair format
parameter_grid_1 = [
    {
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],  # Adjust max_features as needed
        'classifier__n_neighbors': [i for i in range(3, 21, 2)],
        'classifier__p': [1, 2, 3]
    }
]
clf = GridSearchCV(
    estimator=pipe_1, 
    param_grid=parameter_grid_1, 
    scoring='accuracy',
    cv=5,
    return_train_score=True,
    verbose=1
)

# Initialize the auto logger
# max_tuning_runs=None will make sure that all the runs are recorded.
# By default top 5 runs will be recorded for each experiment
mlflow.sklearn.autolog(max_tuning_runs=None)

with mlflow.start_run() as run:
    %time clf.fit(X_train, y_train)

2024/03/29 10:37:32 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 162 candidates, totalling 810 fits
CPU times: total: 22min 6s
Wall time: 18min 35s


In [33]:
pipelines = {
    'knn' : Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', KNeighborsClassifier())
])
, 
    'svc' : Pipeline([
    ('tfidf', TfidfVectorizer()),
        ('classifier', SVC())
    ]),
    'logistic_regression': Pipeline([
    ('tfidf', TfidfVectorizer()),
        ('classifier', LogisticRegression())
    ]),
    'random_forest': Pipeline([
    ('tfidf', TfidfVectorizer()),
        ('classifier', RandomForestClassifier())
    ]),
    'decision_tree': Pipeline([
    ('tfidf', TfidfVectorizer()),
        ('classifier', DecisionTreeClassifier())
    ]),
    'naive_bayes':Pipeline([
    ('tfidf', TfidfVectorizer()),
        ('classifier', GaussianNB())
    ])
}

# Define parameter grid for each algorithm

param_grids = {
    'knn': [
        {
            
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__n_neighbors' : [i for i in range(3, 21, 2)], 
            'classifier__p' : [1, 2, 3]
        }
    ],
    'svc': [
        {
            
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__kernel' : ['rbf'], 
            'classifier__C' : [0.1, 0.01, 1, 10, 100]
        }, 
        {
            
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__kernel' : ['poly'], 
            'classifier__degree' : [2, 3, 4, 5], 
            'classifier__C' : [0.1, 0.01, 1, 10, 100]
        }, 
        {
            
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__kernel' : ['linear'], 
            'classifier__C' : [0.1, 0.01, 1, 10, 100]
        }
    ],
    'logistic_regression': [
        {
            
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__C': [0.1, 1, 10], 
            'classifier__penalty': ['l2']
        }, 
        {
            
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__C': [0.1, 1, 10], 
            'classifier__penalty': ['l1'], 
            'classifier__solver': ['liblinear']
        }, 
        {
           
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__C': [0.1, 1, 10], 
            'classifier__penalty': ['elasticnet'], 
            'classifier__l1_ratio': [0.4, 0.5, 0.6],
            'classifier__solver': ['saga']
        }
    ],
    'random_forest': [
        {
            
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__n_estimators': [50, 100, 200]
        }
    ],
    'decision_tree': [
        {
           
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
            'classifier__max_depth': [None, 5, 10]
        }
    ],
    'naive_bayes': [
        {
           
        'tfidf__ngram_range': [(1, 1), (1, 2)],  # Adjust ngram_range as needed
        'tfidf__max_features': [1000, 2000, 3000],
        }
    ]
}




In [34]:
best_models = {}

# Run the Pipeline
for algo in pipelines.keys():
    print("*"*10, algo, "*"*10)
    grid_search = GridSearchCV(estimator=pipelines[algo], 
                               param_grid=param_grids[algo], 
                               cv=5, 
                               scoring='accuracy', 
                               return_train_score=True,
                               verbose=1
                              )
    
    mlflow.sklearn.autolog(max_tuning_runs=None)
    
    with mlflow.start_run() as run:
        %time grid_search.fit(X_train, y_train)
        
    print('Train Score: ', grid_search.best_score_)
    print('Test Score: ', grid_search.score(X_test, y_test))
    
    best_models[algo] = grid_search.best_estimator_
    print()

********** knn **********


2024/03/29 11:22:02 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 162 candidates, totalling 810 fits
CPU times: total: 21min 55s
Wall time: 17min 46s
Train Score:  0.6486982126506334
Test Score:  0.6339285714285714

********** svc **********


2024/03/29 11:39:45 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 180 candidates, totalling 900 fits
CPU times: total: 1h 35min 4s
Wall time: 1h 40min 53s
Train Score:  0.6659378735918577
Test Score:  0.6602443609022557

********** logistic_regression **********


...

Fitting 5 folds for each of 18 candidates, totalling 90 fits
CPU times: total: 2min 3s
Wall time: 2min 52s
Train Score:  0.6480714988842875
Test Score:  0.6358082706766918

********** naive_bayes **********


2024/03/29 14:36:57 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 5 folds for each of 6 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Internship_2024\Backend_data\ml_flow\.env\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Internship_2024\Backend_data\ml_flow\.env\lib\site-packages\mlflow\utils\autologging_utils\safety.py", line 578, in safe_patch_function
    patch_function(call_original, *args, **kwargs)
  File "e:\Internship_2024\Backend_data\ml_flow\.env\lib\site-packages\mlflow\utils\autologging_utils\safety.py", line 251, in patch_with_managed_run
    result = patch_function(original, *args, **kwargs)
  File "e:\Internship_2024\Backend_data\ml_flow\.env\li

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

# Prefect code


In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [65]:
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X=new_df['Review text']
    y=new_df['Ratings']
    return X, y


In [66]:
def preprocess_data(X_train, X_test, y_train, y_test):
    """
    Rescale the data.
    """
    scaler = TfidfVectorizer()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

In [73]:
def train_model(X_train_scaled, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = KNeighborsClassifier(**hyperparameters)
    clf.fit(X_train_scaled, y_train)
    return clf

In [74]:
def train_model(X_train_scaled, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = KNeighborsClassifier(**hyperparameters)
    clf.fit(X_train_scaled, y_train)
    return clf

In [75]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Compute accuracy scores
    train_score = accuracy_score(y_train, y_train_pred)
    test_score = accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score


In [76]:
new_df.to_csv("New_dataframe.csv")

In [77]:
def workflow(data_path):
    DATA_PATH = data_path
    INPUTS = ['Review text']
    OUTPUT = ['Ratings']
    HYPERPARAMETERS = {'n_neighbors': 3, 'p': 2}
    
    # Load data
    data = pd.read_csv(data_path)

    # Identify Inputs and Output
    X, y = split_inputs_output(data, INPUTS, OUTPUT)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Preprocess the data
    X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(X_train, X_test, y_train, y_test)

    # Build a model
    model = train_model(X_train_scaled, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

In [78]:
if __name__ == "__main__":
    workflow(data_path=r"E:\Internship_2024\Backend_data\ml_flow\New_dataframe.csv")

2024/03/29 16:30:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ea826cbf93a748db82ab4c75ab0bd693', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/03/29 16:30:38 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'


Train Score: 0.6886555938577249
Test Score: 0.6151315789473685


In [82]:
from prefect import task, flow

In [81]:
pip install prefect

     ---------------------------------------- 5.6/5.6 MB 734.2 kB/s eta 0:00:00
     ------------------------------------ 240.7/240.7 KB 984.8 kB/s eta 0:00:00
     -------------------------------------- 125.0/125.0 KB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 60.5/60.5 KB 1.6 MB/s eta 0:00:00
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached pendulum-2.1.2-cp310-cp310-win_amd64.whl
     -------------------------------------- 295.0/295.0 KB 1.2 MB/s eta 0:00:00
     -------------------------------------- 43.6/43.6 KB 709.3 kB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 498.6 kB/s eta 0:00:00
     -------------------------------------- 139.2/139.2 KB 1.0 MB/s eta 0:00:00
     ------------------------------------ 553.1/553.1 KB 937.5 kB/s eta 0:00:00
     -------------------------------------- 85.5/85.5 KB 965.2 kB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 890.2 kB/s eta 0:00:00
     ---

You should consider upgrading via the 'e:\Internship_2024\Backend_data\ml_flow\.env\Scripts\python.exe -m pip install --upgrade pip' command.


In [83]:
from prefect import task, flow

In [84]:
@task
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)


@task
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y
	

@task
def split_train_test(X, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)
	
	
@task
def preprocess_data(X_train, X_test, y_train, y_test):
    """
    Rescale the data.
    """
    scaler = TfidfVectorizer()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test
	

@task
def train_model(X_train_scaled, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = KNeighborsClassifier(**hyperparameters)
    clf.fit(X_train_scaled, y_train)
    return clf
	

@task
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Compute accuracy scores
    train_score = accuracy_score(y_train, y_train_pred)
    test_score = accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score


In [91]:

@flow(name="KNN Training Flow")
def workflow():
    DATA_PATH = r"E:\Internship_2024\Backend_data\ml_flow\New_dataframe.csv"
    INPUTS = 'Review text'
    OUTPUT = 'Ratings'
    HYPERPARAMETERS = {'n_neighbors': 3, 'p': 2}
    
    # Load data
    data = load_data(DATA_PATH)

    # Identify Inputs and Output
    X, y = split_inputs_output(data, INPUTS, OUTPUT)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_test(X, y)

    # Preprocess the data
    X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(X_train, X_test, y_train, y_test)

    # Build a model
    model = train_model(X_train_scaled, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

if __name__ == "__main__":
    workflow.serve(
        name="my-first-deployment",
        cron="* * * * *"
    )

In [92]:
if __name__ == "__main__":
    workflow()

16:57:28.767 | INFO    | prefect.engine - Created flow run 'pink-frigatebird' for flow 'KNN Training Flow'

16:57:30.380 | INFO    | Flow run 'pink-frigatebird' - Created task run 'load_data-0' for task 'load_data'

16:57:30.386 | INFO    | Flow run 'pink-frigatebird' - Executing 'load_data-0' immediately...

16:57:40.327 | INFO    | Task run 'load_data-0' - Finished in state Completed()

16:57:40.577 | INFO    | Flow run 'pink-frigatebird' - Created task run 'split_inputs_output-0' for task 'split_inputs_output'

...

16:59:15.894 | INFO    | Flow run 'pink-frigatebird' - Created task run 'evaluate_model-0' for task 'evaluate_model'

16:59:15.898 | INFO    | Flow run 'pink-frigatebird' - Executing 'evaluate_model-0' immediately...

16:59:20.133 | INFO    | Task run 'evaluate_model-0' - Finished in state Completed()

Train Score: 0.6881855217800062
Test Score: 0.5709586466165414


16:59:20.511 | INFO    | Flow run 'pink-frigatebird' - Finished in state Completed('All states completed.')